In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
df = pd.read_csv('ITIM_Format_Train.csv')
df_test = pd.read_csv('ITIM_Format_Test.csv')
df_test.set_index('PassengerId',inplace=True)

In [3]:
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Titles,Number_Names,Ticket Prefix,male,Q,S
PassengerId,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0,3,0,1,1,0
893,3,47.0,1,0,7.0000,0,5,0,0,0,1
894,2,62.0,0,0,9.6875,0,4,0,1,1,0
895,3,27.0,0,0,8.6625,0,3,0,1,0,1
896,3,22.0,1,1,12.2875,0,6,0,0,0,1


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train = df.drop(['Survived'],axis=1)
y_train = df['Survived']

X_test = df_test

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
RFC = RandomForestClassifier(n_estimators=100)

In [8]:
RFC.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
predictions = RFC.predict(X_test)

In [10]:
df_test['Survived'] = predictions

In [11]:
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Titles,Number_Names,Ticket Prefix,male,Q,S,Survived
PassengerId,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0,3,0,1,1,0,0
893,3,47.0,1,0,7.0000,0,5,0,0,0,1,0
894,2,62.0,0,0,9.6875,0,4,0,1,1,0,0
895,3,27.0,0,0,8.6625,0,3,0,1,0,1,1
896,3,22.0,1,1,12.2875,0,6,0,0,0,1,1


In [12]:
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Titles,Number_Names,Ticket Prefix,male,Q,S,Survived
PassengerId,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0,3,0,1,1,0,0
893,3,47.0,1,0,7.0000,0,5,0,0,0,1,0
894,2,62.0,0,0,9.6875,0,4,0,1,1,0,0
895,3,27.0,0,0,8.6625,0,3,0,1,0,1,1
896,3,22.0,1,1,12.2875,0,6,0,0,0,1,1


In [13]:
cols = list(df_test.drop(['Survived'],axis=1).columns)

In [14]:
submission = df_test.drop(cols,axis=1)

In [15]:
submission.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,1
896,1


In [16]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 1 columns):
Survived    418 non-null int64
dtypes: int64(1)
memory usage: 6.5 KB


In [17]:
submission.to_csv(r'Submission_v1.csv', index=True)

In [10]:
from sklearn.metrics import classification_report,confusion_matrix

In [53]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[151   9]
 [ 27  80]]


              precision    recall  f1-score   support

           0       0.85      0.94      0.89       160
           1       0.90      0.75      0.82       107

    accuracy                           0.87       267
   macro avg       0.87      0.85      0.85       267
weighted avg       0.87      0.87      0.86       267



In [18]:
#Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

#Number of trees
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [19]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [20]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100,
                               cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [22]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': False}

In [32]:
hp_predictions = rf_random.predict(X_test)

In [43]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [80, 90, 100, 110]
}
# Instantiate the grid search model
grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, 
                          cv=3, n_jobs=-1,verbose=2)

In [44]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  1.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [45]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

In [46]:
grid_predictions = grid_search.predict(X_test)

In [47]:
#Original Model
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[151   9]
 [ 27  80]]


              precision    recall  f1-score   support

           0       0.85      0.94      0.89       160
           1       0.90      0.75      0.82       107

    accuracy                           0.87       267
   macro avg       0.87      0.85      0.85       267
weighted avg       0.87      0.87      0.86       267



In [48]:
#Hyperparameter Tuned Model (Random Search)
print(confusion_matrix(y_test,hp_predictions))
print('\n')
print(classification_report(y_test,hp_predictions))

[[148  12]
 [ 28  79]]


              precision    recall  f1-score   support

           0       0.84      0.93      0.88       160
           1       0.87      0.74      0.80       107

    accuracy                           0.85       267
   macro avg       0.85      0.83      0.84       267
weighted avg       0.85      0.85      0.85       267



In [49]:
#Hyperparameter Tuned Model (Grid Search)
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))

[[144  16]
 [ 26  81]]


              precision    recall  f1-score   support

           0       0.85      0.90      0.87       160
           1       0.84      0.76      0.79       107

    accuracy                           0.84       267
   macro avg       0.84      0.83      0.83       267
weighted avg       0.84      0.84      0.84       267



In [50]:
from sklearn.metrics import accuracy_score

In [52]:
print(accuracy_score(y_test,predictions))

0.8651685393258427
